In [20]:
from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
# from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import applications
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import h5py
import os
import json
import pickle as cPickle
import datetime
import time

In [2]:
img_width, img_height = 224,224

In [3]:
model = applications.InceptionV3(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))

In [14]:
inceptionv3=Model(input=model.input, output=model.get_layer('mixed10').output)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("mi..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [5]:
from keras.models import model_from_json

In [7]:
import os

In [8]:
os.mkdir("model_json")

In [15]:
# serialize model to JSON
model_json = inceptionv3.to_json()
with open("model_json/inceptionv3.json", "w") as json_file:
    json_file.write(model_json)

In [10]:
### For fine tune
# Top Model Block
'''
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(nb_classes, activation='softmax')(x)

    # add your top layer block to your base model
    model = Model(base_model.input, predictions)
    print(model.summary())

    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train
    # # uncomment when choosing based_model_last_block_layer
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all layers of the based model that is already pre-trained.
    for layer in base_model.layers:
        layer.trainable = False
'''

"\n    x = base_model.output\n    x = GlobalAveragePooling2D()(x)\n    predictions = Dense(nb_classes, activation='softmax')(x)\n\n    # add your top layer block to your base model\n    model = Model(base_model.input, predictions)\n    print(model.summary())\n\n    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train\n    # # uncomment when choosing based_model_last_block_layer\n    # for i, layer in enumerate(model.layers):\n    #     print(i, layer.name)\n\n    # first: train only the top layers (which were randomly initialized)\n    # i.e. freeze all layers of the based model that is already pre-trained.\n    for layer in base_model.layers:\n        layer.trainable = False\n"

In [12]:
model_ = applications.VGG16(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))
vgg=Model(input=model_.input, output=model_.get_layer('block5_pool').output)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  


In [13]:
# serialize model to JSON
model_json = vgg.to_json()
with open("model_json/vgg16.json", "w") as json_file:
    json_file.write(model_json)

In [17]:
xception =applications.Xception(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))
feat_ext_xception=Model(input=xception.input, output=xception.get_layer("block14_sepconv2_act").output)
# serialize model to JSON
model_json = feat_ext_xception.to_json()
with open("model_json/xception.json", "w") as json_file:
    json_file.write(model_json)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  


In [18]:
## After save the models

In [1]:
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import h5py
import os
import json
import pickle as cPickle
import datetime
import time

Using TensorFlow backend.


In [2]:
img_width, img_height = 224,224

In [15]:
img_size=224

In [3]:
train_data_dir = "data/MSTAR_Chips/TRAIN_images"
validation_data_dir = "data/MSTAR_Chips/TEST_images"
nb_train_samples = 1622
nb_validation_samples = 1365 
batch_size = 32
epochs = 50

In [4]:
# path to training dataset
train_labels = os.listdir(train_data_dir)

In [11]:
import os
import glob

models=[]
model_names=[]
json_dir_name = "model_json"
weight_name=""
json_pattern = os.path.join(json_dir_name,'*.json')
file_list = glob.glob(json_pattern)
for file in file_list:
    print("Json file path :",file)
    # load json and create model
    json_file = open(file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    weight_name= file.split(".")
    print("Weight path : {}.h5".format(weight_name[0]))
    #load weights into new model
    loaded_model.load_weights("{}.h5".format(weight_name[0]))
    models.append(loaded_model)  
    model_names.append(weight_name[0].split("/")[-1])

Json file path : model_json/inceptionv3.json
Weight path : model_json/inceptionv3.h5
Json file path : model_json/vgg16.json
Weight path : model_json/vgg16.h5
Json file path : model_json/xception.json
Weight path : model_json/xception.h5


In [13]:
model_names=["inceptionv3","vgg16","xception"]

In [12]:
import h5py

In [ ]:
image_size=(224,224)
h5f.close()
for model,name in zip(models,model_names):
    # variables to hold features and labels
    features = []
    labels = []
    test_features = []
    test_labels = []
    # h5f = h5py.File('features/{}.h5'.format(name), 'w')
    with h5py.File('features/{}.hdf5'.format(name),'w') as f:
        # loop over all the labels in the folder
        for label in train_labels:
            cur_path = train_data_dir + "/" + label
            print(cur_path)
            for image_path in glob.glob(cur_path + "/*.jpg"):
                img = image.load_img(image_path, target_size=image_size)
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
                feature = model.predict(x)
                flat = feature.flatten()
                features.append(flat)
                labels.append(label)
        f.create_dataset('train', data=features)
        f.create_dataset('train_label', data=labels)
        # loop over all the labels in the folder
        for label in train_labels:
            cur_path = validation_data_dir + "/" + label
            for image_path in glob.glob(cur_path + "/*.jpg"):
                img = image.load_img(image_path, target_size=image_size)
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
                feature = feat_ext_vgg.predict(x)
                flat = feature.flatten()
                test_features.append(flat)
                test_labels.append(label)
        f.create_dataset('test', data=test_features)
        f.create_dataset('test_label', data=test_labels)
    #h5f.close()

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank
